In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Markdown, Math

seed = 42

_Частина 3_

Завдання: розробити програмне забезпечення для реалізації двошарового персептрону із структурою 2-3-1.

Передбачити режим навчання «ON-LINE» та режим розпізнавання.

Піддослідна функція $х_1+х_2=у$

In [16]:
class NeuralNetwork:
    """
    N-layer neural network mathematic model with sigmoid activation function
    """

    def __init__(self, start_vector, result, npl, add_bias=True):
        """
        Init starting values, first step
        
        Args:
            start_vector: input vector
            result: predicted result after training
            npl: Neuron-per-Layer (i.e. 2-3-1 == [2, 3, 1])
        """
        self._npl = npl
        self.add_bias = add_bias
        # First_neuron_number . depends on bias
        self.one_or_zero = 1 if self.add_bias else 0
        self.one_or_none = 1 if self.add_bias else None
        # Add bias neuron if it needed
        self._inputs = np.concatenate((self.one_or_none, start_vector))
        self._result = np.array(result)
        self.weights = []
        for i in range(len(npl)-1):
            w = np.random.default_rng(seed).uniform(
                low=0.3,
                high=0.5,
                size=(npl[i]+self.one_or_zero)*npl[i+1],
            ).round(3).reshape((npl[i]+self.one_or_zero, npl[i+1]))
            self.weights.append(w)
        print(self.weights)
        input()
        self.neurons = []
        self.neurons.append(self._inputs)
        for i in range(1, len(npl)):
            self.neurons.append(np.concatenate((self.one_or_none, np.empty(npl[i]))))
        
        self.epoch_history = []  # same as y_i # but this time it will be in each neuron
        self.error_history = []  # same as dn_i
        self._step_forward()
        

    def sigm(self, x):
        """
        Sigmoidal activation function
        """
        return 1 / (1 + np.exp(-x))

    def sigm_der(self, w):
        """
        Args:
            w: the previous output value of the neuron
        """
        return w * (1 - w)

    def _step_forward(self):
        """
        Make one calculation forward through all NN
        """
        for i in range(1, len(self._npl)):
            # TODO: check this multiplication
            print("step forward", i)
            self.neurons[i] = np.dot(self.neurons[i-1][self.one_or_zero:], self.weights[-1])
            
        self.epoch_history.append(
            self.sigm(np.dot(self._inputs, self.weights)))
        self.error_history.append(
            np.abs((self._result - self.epoch_history[-1]) / self._result)
        )

    def _back_propagate(self, learning_rate):
        """
        Correct previous iteration
        """
        l = learning_rate
        # that's a local gradient
        #         |-------------e-----------------------| * |------------derivative--------------|     
        delta_w = (self._result - self.epoch_history[-1]) * self.sigm_der(self.epoch_history[-1])
        #                  |--out signal--| |------|
        correction = np.dot(self._inputs.T, delta_w*l)
        self.weights += correction

    def train(self, acceptable_error=0.01, learning_rate=0.1, max_epoch_iteration=50000):
        """
        Train function
        Make forward step and correct weights.
        Loop that untill error will be small enough.
        Limit maximum number of epochs.
        """
        i = 1
        while (self.error_history[-1] > acceptable_error) and i < max_epoch_iteration:
            self._back_propagate(learning_rate)
            self._step_forward()
            i += 1
    
    def predict(self, input_vector):
        """
        Predict input vector with current weights
        """
        return self.sigm(np.dot(input_vector, self.weights))

In [17]:
perceptron = NeuralNetwork([2,2], [4], [2, 3, 1])

[array([[0.455, 0.388, 0.472],
       [0.439, 0.319, 0.495],
       [0.452, 0.457, 0.326]]), array([[0.455],
       [0.388],
       [0.472],
       [0.439]])]

step forward 1


ValueError: shapes (2,) and (4,1) not aligned: 2 (dim 0) != 4 (dim 0)